In [16]:
import glob
import os
import re
import datetime
import subprocess
import shlex
import gdal

import downloader
import af_import
import rst_import_lct
import rst_import_vcf
import rst_import
import polygon_import
import run_step1
import run_step2
import export_shp

## ActiveFire data

In [5]:

tag_af = 'mod_na_2012'

# source dir
ddir = 'downloads/firms'

# shp file names
arcnames = ['M6_34602', ]
af_fnames = [os.path.join(ddir, tag_af, 'fire_archive_%s.shp' % _) for _ in arcnames]


for fn in af_fnames:
    if not os.path.exists(fn):
        zn = list(os.path.split(fn))
        #print(zn)
        zn[1] = re.sub(r'fire_archive_([^.]+).shp', r'DL_FIRE_\1.zip', zn[1])
        #print(zn)
        zn = os.path.join(*zn)
        #print(zn)
        subprocess.run(['unzip', zn, '-d', os.path.join(ddir, tag_af)])

## Download MODIS land cover and vegetation fields data

In [6]:
year = 2012
is_leap = (year % 4 == 0)
tag_lct = 'modlct_%d' % year
tag_vcf = 'modvcf_%d' % year
tag_regnum = 'regnum'

In [7]:
url_lct = 'https://e4ftl01.cr.usgs.gov/MOTA/MCD12Q1.006/%d.01.01/' % year
url_vcf = 'https://e4ftl01.cr.usgs.gov/MOLT/MOD44B.006/%d.03.%02d/' % (year, 5 if is_leap else 6)

ddir_lct = './downloads/e4ftl01.cr.usgs.gov/MOTA/MCD12Q1.006/%d.01.01' % year
ddir_vcf = './downloads/e4ftl01.cr.usgs.gov/MOLT/MOD44B.006/%d.03.%02d/' % (year, 5 if is_leap else 6)

In [26]:
from importlib import reload
reload(downloader)
downloader.download_only_needed(url_lct, pnts=af_fnames[0])

reading shp file ... Done
<class 'numpy.ndarray'>
False


/home/finn/code_anaconda/downloader.py:110: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 110 of the file /home/finn/code_anaconda/downloader.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  soup = BeautifulSoup(requests.get(url).text)


https://e4ftl01.cr.usgs.gov/MOTA/MCD12Q1.006/2012.01.01/BROWSE.MCD12Q1.A2012001.h07v06.006.2018054210306.1.jpg
https://e4ftl01.cr.usgs.gov/MOTA/MCD12Q1.006/2012.01.01/BROWSE.MCD12Q1.A2012001.h07v07.006.2018054211946.1.jpg
https://e4ftl01.cr.usgs.gov/MOTA/MCD12Q1.006/2012.01.01/BROWSE.MCD12Q1.A2012001.h08v04.006.2018054212230.1.jpg
https://e4ftl01.cr.usgs.gov/MOTA/MCD12Q1.006/2012.01.01/BROWSE.MCD12Q1.A2012001.h08v05.006.2018054212715.1.jpg
https://e4ftl01.cr.usgs.gov/MOTA/MCD12Q1.006/2012.01.01/BROWSE.MCD12Q1.A2012001.h08v06.006.2018054213207.1.jpg
https://e4ftl01.cr.usgs.gov/MOTA/MCD12Q1.006/2012.01.01/BROWSE.MCD12Q1.A2012001.h08v07.006.2018054213232.1.jpg
https://e4ftl01.cr.usgs.gov/MOTA/MCD12Q1.006/2012.01.01/BROWSE.MCD12Q1.A2012001.h09v03.006.2018054214637.1.jpg
https://e4ftl01.cr.usgs.gov/MOTA/MCD12Q1.006/2012.01.01/BROWSE.MCD12Q1.A2012001.h09v04.006.2018054215001.1.jpg
https://e4ftl01.cr.usgs.gov/MOTA/MCD12Q1.006/2012.01.01/BROWSE.MCD12Q1.A2012001.h09v05.006.2018054215129.1.jpg
h

https://e4ftl01.cr.usgs.gov/MOTA/MCD12Q1.006/2012.01.01/MCD12Q1.A2012001.h12v07.006.2018054231348.hdf
https://e4ftl01.cr.usgs.gov/MOTA/MCD12Q1.006/2012.01.01/MCD12Q1.A2012001.h12v07.006.2018054231348.hdf.xml
https://e4ftl01.cr.usgs.gov/MOTA/MCD12Q1.006/2012.01.01/MCD12Q1.A2012001.h13v03.006.2018055002216.hdf
https://e4ftl01.cr.usgs.gov/MOTA/MCD12Q1.006/2012.01.01/MCD12Q1.A2012001.h13v03.006.2018055002216.hdf.xml
https://e4ftl01.cr.usgs.gov/MOTA/MCD12Q1.006/2012.01.01/MCD12Q1.A2012001.h13v04.006.2018054235139.hdf
https://e4ftl01.cr.usgs.gov/MOTA/MCD12Q1.006/2012.01.01/MCD12Q1.A2012001.h13v04.006.2018054235139.hdf.xml
https://e4ftl01.cr.usgs.gov/MOTA/MCD12Q1.006/2012.01.01/MCD12Q1.A2012001.h14v03.006.2018055001956.hdf
https://e4ftl01.cr.usgs.gov/MOTA/MCD12Q1.006/2012.01.01/MCD12Q1.A2012001.h14v03.006.2018055001956.hdf.xml
https://e4ftl01.cr.usgs.gov/MOTA/MCD12Q1.006/2012.01.01/MCD12Q1.A2012001.h14v04.006.2018055002011.hdf
https://e4ftl01.cr.usgs.gov/MOTA/MCD12Q1.006/2012.01.01/MCD12Q1.A2

In [27]:
downloader.purge_corrupted(ddir_lct, url=url_lct)

./downloads/e4ftl01.cr.usgs.gov/MOTA/MCD12Q1.006/2012.01.01/MCD12Q1.A2012001.h07v06.006.2018054210306.hdf ... OK
./downloads/e4ftl01.cr.usgs.gov/MOTA/MCD12Q1.006/2012.01.01/MCD12Q1.A2012001.h07v07.006.2018054211946.hdf ... OK
./downloads/e4ftl01.cr.usgs.gov/MOTA/MCD12Q1.006/2012.01.01/MCD12Q1.A2012001.h08v04.006.2018054212230.hdf ... OK
./downloads/e4ftl01.cr.usgs.gov/MOTA/MCD12Q1.006/2012.01.01/MCD12Q1.A2012001.h08v05.006.2018054212715.hdf ... OK
./downloads/e4ftl01.cr.usgs.gov/MOTA/MCD12Q1.006/2012.01.01/MCD12Q1.A2012001.h08v06.006.2018054213207.hdf ... OK
./downloads/e4ftl01.cr.usgs.gov/MOTA/MCD12Q1.006/2012.01.01/MCD12Q1.A2012001.h08v07.006.2018054213232.hdf ... OK
./downloads/e4ftl01.cr.usgs.gov/MOTA/MCD12Q1.006/2012.01.01/MCD12Q1.A2012001.h09v03.006.2018054214637.hdf ... OK
./downloads/e4ftl01.cr.usgs.gov/MOTA/MCD12Q1.006/2012.01.01/MCD12Q1.A2012001.h09v04.006.2018054215001.hdf ... OK
./downloads/e4ftl01.cr.usgs.gov/MOTA/MCD12Q1.006/2012.01.01/MCD12Q1.A2012001.h09v05.006.20180542

In [29]:
from importlib import reload
reload(downloader)
downloader.download_only_needed(url_vcf, pnts=af_fnames[0])

reading shp file ... Done
https://e4ftl01.cr.usgs.gov/MOLT/MOD44B.006/2012.03.05/BROWSE.MOD44B.A2012065.h07v06.006.2017081132640.1.jpg
https://e4ftl01.cr.usgs.gov/MOLT/MOD44B.006/2012.03.05/BROWSE.MOD44B.A2012065.h07v07.006.2017081130654.1.jpg
https://e4ftl01.cr.usgs.gov/MOLT/MOD44B.006/2012.03.05/BROWSE.MOD44B.A2012065.h08v04.006.2017081122051.1.jpg
https://e4ftl01.cr.usgs.gov/MOLT/MOD44B.006/2012.03.05/BROWSE.MOD44B.A2012065.h08v05.006.2017081125522.1.jpg
https://e4ftl01.cr.usgs.gov/MOLT/MOD44B.006/2012.03.05/BROWSE.MOD44B.A2012065.h08v06.006.2017081134816.1.jpg
https://e4ftl01.cr.usgs.gov/MOLT/MOD44B.006/2012.03.05/BROWSE.MOD44B.A2012065.h08v07.006.2017081132004.1.jpg
https://e4ftl01.cr.usgs.gov/MOLT/MOD44B.006/2012.03.05/BROWSE.MOD44B.A2012065.h09v03.006.2017081131529.1.jpg
https://e4ftl01.cr.usgs.gov/MOLT/MOD44B.006/2012.03.05/BROWSE.MOD44B.A2012065.h09v04.006.2017081130241.1.jpg
https://e4ftl01.cr.usgs.gov/MOLT/MOD44B.006/2012.03.05/BROWSE.MOD44B.A2012065.h09v05.006.2017081130345

https://e4ftl01.cr.usgs.gov/MOLT/MOD44B.006/2012.03.05/MOD44B.A2012065.h13v03.006.2017081171025.hdf
https://e4ftl01.cr.usgs.gov/MOLT/MOD44B.006/2012.03.05/MOD44B.A2012065.h13v03.006.2017081171025.hdf.xml
https://e4ftl01.cr.usgs.gov/MOLT/MOD44B.006/2012.03.05/MOD44B.A2012065.h13v04.006.2017081171441.hdf
https://e4ftl01.cr.usgs.gov/MOLT/MOD44B.006/2012.03.05/MOD44B.A2012065.h13v04.006.2017081171441.hdf.xml
https://e4ftl01.cr.usgs.gov/MOLT/MOD44B.006/2012.03.05/MOD44B.A2012065.h14v03.006.2017081171107.hdf
https://e4ftl01.cr.usgs.gov/MOLT/MOD44B.006/2012.03.05/MOD44B.A2012065.h14v03.006.2017081171107.hdf.xml
https://e4ftl01.cr.usgs.gov/MOLT/MOD44B.006/2012.03.05/MOD44B.A2012065.h14v04.006.2017081171010.hdf
https://e4ftl01.cr.usgs.gov/MOLT/MOD44B.006/2012.03.05/MOD44B.A2012065.h14v04.006.2017081171010.hdf.xml


In [30]:
downloader.purge_corrupted(ddir_vcf, url=url_vcf)

./downloads/e4ftl01.cr.usgs.gov/MOLT/MOD44B.006/2012.03.05/MOD44B.A2012065.h07v06.006.2017081170142.hdf ... OK
./downloads/e4ftl01.cr.usgs.gov/MOLT/MOD44B.006/2012.03.05/MOD44B.A2012065.h07v07.006.2017081170138.hdf ... OK
./downloads/e4ftl01.cr.usgs.gov/MOLT/MOD44B.006/2012.03.05/MOD44B.A2012065.h08v04.006.2017081160719.hdf ... OK
./downloads/e4ftl01.cr.usgs.gov/MOLT/MOD44B.006/2012.03.05/MOD44B.A2012065.h08v05.006.2017081160744.hdf ... OK
./downloads/e4ftl01.cr.usgs.gov/MOLT/MOD44B.006/2012.03.05/MOD44B.A2012065.h08v06.006.2017081170149.hdf ... OK
./downloads/e4ftl01.cr.usgs.gov/MOLT/MOD44B.006/2012.03.05/MOD44B.A2012065.h08v07.006.2017081170236.hdf ... OK
./downloads/e4ftl01.cr.usgs.gov/MOLT/MOD44B.006/2012.03.05/MOD44B.A2012065.h09v03.006.2017081170410.hdf ... OK
./downloads/e4ftl01.cr.usgs.gov/MOLT/MOD44B.006/2012.03.05/MOD44B.A2012065.h09v04.006.2017081160713.hdf ... OK
./downloads/e4ftl01.cr.usgs.gov/MOLT/MOD44B.006/2012.03.05/MOD44B.A2012065.h09v05.006.2017081160758.hdf ... OK
.

## Create a PostGIS database

In [8]:
os.environ['PGDATABASE'] = 'gis'
os.environ['PGUSER'] = 'docker'
os.environ['PGPASSWORD'] = 'docker'
os.environ['PGHOST'] = 'localhost'
os.environ['PGPORT'] = '5432'

In [9]:
!psql postgres -c 'SELECT version();'
!pg_lsclusters

                                                             version                                                              
----------------------------------------------------------------------------------------------------------------------------------
 PostgreSQL 10.6 (Debian 10.6-1.pgdg90+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 6.3.0-18+deb9u1) 6.3.0 20170516, 64-bit
(1 row)

Ver Cluster Port Status Owner    Data directory              Log file
10  main    5432 online postgres /var/lib/postgresql/10/main /var/log/postgresql/postgresql-10-main.log
11  main    5433 down   postgres /var/lib/postgresql/11/main /var/log/postgresql/postgresql-11-main.log


In [10]:
!psql -d gis -c 'CREATE LANGUAGE plpython3u;' 

CREATE LANGUAGE


## Import land cover type

Get filenames of MCD12Q1 land cover type data

In [25]:
# tag to identify dataset
ddir_lct = './downloads/e4ftl01.cr.usgs.gov/MOTA/MCD12Q1.006'

search_string = "%(ddir)s/%(year)s.01.01/MCD12Q1.A%(year)s001.h??v??.006.*.hdf" % dict(
        ddir = ddir_lct, year=year)

In [26]:
# grab hdf file names
fnames = sorted(glob.glob(search_string))
print('found %d hdf files' % len(fnames) )

found 29 hdf files


In [146]:
from importlib import reload
reload(rst_import)
#rst_import_lct.main(tag_lct, fnames, run_merge=False)
rst_import.main(tag_lct, 'lct', fnames, run_merge=False, run_resample=False, run_import=True)

bname: MCD12Q1.A2012001
1 of 77...
2 of 77...
3 of 77...
4 of 77...
5 of 77...
6 of 77...
7 of 77...
8 of 77...
9 of 77...
10 of 77...
11 of 77...
12 of 77...
13 of 77...
14 of 77...
15 of 77...
16 of 77...
17 of 77...
18 of 77...
19 of 77...
20 of 77...
21 of 77...
22 of 77...
23 of 77...
24 of 77...
25 of 77...
26 of 77...
27 of 77...
28 of 77...
29 of 77...
30 of 77...
31 of 77...
32 of 77...
33 of 77...
34 of 77...
35 of 77...
36 of 77...
37 of 77...
38 of 77...
39 of 77...
40 of 77...
41 of 77...
42 of 77...
43 of 77...
44 of 77...
45 of 77...
46 of 77...
47 of 77...
48 of 77...
49 of 77...
50 of 77...
51 of 77...
52 of 77...
53 of 77...
54 of 77...
55 of 77...
56 of 77...
57 of 77...
58 of 77...
59 of 77...
60 of 77...
61 of 77...
62 of 77...
63 of 77...
64 of 77...
65 of 77...
66 of 77...
67 of 77...
68 of 77...
69 of 77...
70 of 77...
71 of 77...
72 of 77...
73 of 77...
74 of 77...
75 of 77...
76 of 77...
77 of 77...
['shp2pgsql', '-d', '-s', '4326', '-I', 'tmp_skel.shp', 'rast

## Import vegetation continuous fields

In [147]:
ddir = './downloads/e4ftl01.cr.usgs.gov/MOLT/MOD44B.006'

search_string = "%(ddir)s/%(year)s.03.0[56]/MOD44B.A%(year)s065.h??v??.006.*.hdf" % dict(
        ddir = ddir, year=year)

# grab hdf file names
fnames = sorted(glob.glob(search_string))

In [157]:
from importlib import reload
reload(rst_import)
rst_import.main(tag_vcf, 'vcf', fnames)

bname: MOD44B.A2012065
cmd: gdal_merge.py -separate -o ./proc_modvcf_2012/mrg/MOD44B.A2012065.h07v06.006.2017081170142.tif -co COMPRESS=LZW -co TILED=YES HDF4_EOS:EOS_GRID:"./downloads/e4ftl01.cr.usgs.gov/MOLT/MOD44B.006/2012.03.05/MOD44B.A2012065.h07v06.006.2017081170142.hdf":MOD44 ...
cmd: gdal_merge.py -separate -o ./proc_modvcf_2012/mrg/MOD44B.A2012065.h07v07.006.2017081170138.tif -co COMPRESS=LZW -co TILED=YES HDF4_EOS:EOS_GRID:"./downloads/e4ftl01.cr.usgs.gov/MOLT/MOD44B.006/2012.03.05/MOD44B.A2012065.h07v07.006.2017081170138.hdf":MOD44 ...
cmd: gdal_merge.py -separate -o ./proc_modvcf_2012/mrg/MOD44B.A2012065.h08v04.006.2017081160719.tif -co COMPRESS=LZW -co TILED=YES HDF4_EOS:EOS_GRID:"./downloads/e4ftl01.cr.usgs.gov/MOLT/MOD44B.006/2012.03.05/MOD44B.A2012065.h08v04.006.2017081160719.hdf":MOD44 ...
cmd: gdal_merge.py -separate -o ./proc_modvcf_2012/mrg/MOD44B.A2012065.h08v05.006.2017081160744.tif -co COMPRESS=LZW -co TILED=YES HDF4_EOS:EOS_GRID:"./downloads/e4ftl01.cr.usgs.gov/

cmd: gdalwarp -t_srs "+proj=longlat +datum=WGS84 +no_defs" -tr 0.00166666666666666666666666666667 0.00166666666666666666666666666667 -te -180 30 -170 40 -overwrite -r average -dstnodata 255 -wo INIT_DEST=NO_DATA -wo NUM_THREADS=ALL_CPUS -co COMPRESS=LZW -co TI ...
cmd: gdalwarp -t_srs "+proj=longlat +datum=WGS84 +no_defs" -tr 0.00166666666666666666666666666667 0.00166666666666666666666666666667 -te -180 20 -170 30 -overwrite -r average -dstnodata 255 -wo INIT_DEST=NO_DATA -wo NUM_THREADS=ALL_CPUS -co COMPRESS=LZW -co TI ...
cmd: gdalwarp -t_srs "+proj=longlat +datum=WGS84 +no_defs" -tr 0.00166666666666666666666666666667 0.00166666666666666666666666666667 -te -180 10 -170 20 -overwrite -r average -dstnodata 255 -wo INIT_DEST=NO_DATA -wo NUM_THREADS=ALL_CPUS -co COMPRESS=LZW -co TI ...
cmd: gdalwarp -t_srs "+proj=longlat +datum=WGS84 +no_defs" -tr 0.00166666666666666666666666666667 0.00166666666666666666666666666667 -te -170 50 -160 60 -overwrite -r average -dstnodata 255 -wo INIT_DEST=N

cmd: gdalwarp -t_srs "+proj=longlat +datum=WGS84 +no_defs" -tr 0.00166666666666666666666666666667 0.00166666666666666666666666666667 -te -120 20 -110 30 -overwrite -r average -dstnodata 255 -wo INIT_DEST=NO_DATA -wo NUM_THREADS=ALL_CPUS -co COMPRESS=LZW -co TI ...
cmd: gdalwarp -t_srs "+proj=longlat +datum=WGS84 +no_defs" -tr 0.00166666666666666666666666666667 0.00166666666666666666666666666667 -te -120 10 -110 20 -overwrite -r average -dstnodata 255 -wo INIT_DEST=NO_DATA -wo NUM_THREADS=ALL_CPUS -co COMPRESS=LZW -co TI ...
cmd: gdalwarp -t_srs "+proj=longlat +datum=WGS84 +no_defs" -tr 0.00166666666666666666666666666667 0.00166666666666666666666666666667 -te -120 0 -110 10 -overwrite -r average -dstnodata 255 -wo INIT_DEST=NO_DATA -wo NUM_THREADS=ALL_CPUS -co COMPRESS=LZW -co TIL ...
cmd: gdalwarp -t_srs "+proj=longlat +datum=WGS84 +no_defs" -tr 0.00166666666666666666666666666667 0.00166666666666666666666666666667 -te -110 50 -100 60 -overwrite -r average -dstnodata 255 -wo INIT_DEST=N

cmd: gdalwarp -t_srs "+proj=longlat +datum=WGS84 +no_defs" -tr 0.00166666666666666666666666666667 0.00166666666666666666666666666667 -te -70 10 -60 20 -overwrite -r average -dstnodata 255 -wo INIT_DEST=NO_DATA -wo NUM_THREADS=ALL_CPUS -co COMPRESS=LZW -co TILE ...
cmd: gdalwarp -t_srs "+proj=longlat +datum=WGS84 +no_defs" -tr 0.00166666666666666666666666666667 0.00166666666666666666666666666667 -te -70 0 -60 10 -overwrite -r average -dstnodata 255 -wo INIT_DEST=NO_DATA -wo NUM_THREADS=ALL_CPUS -co COMPRESS=LZW -co TILED ...
cmd: gdalwarp -t_srs "+proj=longlat +datum=WGS84 +no_defs" -tr 0.00166666666666666666666666666667 0.00166666666666666666666666666667 -te -60 50 -50 60 -overwrite -r average -dstnodata 255 -wo INIT_DEST=NO_DATA -wo NUM_THREADS=ALL_CPUS -co COMPRESS=LZW -co TILE ...
cmd: gdalwarp -t_srs "+proj=longlat +datum=WGS84 +no_defs" -tr 0.00166666666666666666666666666667 0.00166666666666666666666666666667 -te -60 40 -50 50 -overwrite -r average -dstnodata 255 -wo INIT_DEST=NO_

## Import countries of the world shapefile

In [158]:
if not os.path.exists('all_countries/All_Countries.shp'):
    subprocess.run(shlex.split('wget https://s3-us-west-2.amazonaws.com/earthlab-finn/All_Countries.zip'))
    subprocess.run(shlex.split('unzip All_Countries.zip -d all_countries'))
    subprocess.run(['rm', 'All_Countries.zip'])

In [159]:
polygon_import.main('regnum')

['shp2pgsql', '-d', '-c', '-s', '4326', '-I', 'all_countries/All_Countries.shp', 'raster.rst_regnum']
(None, None)


## Import active fire data

In [160]:
dt0 = datetime.date(year, 1, 1)
dt1 = datetime.date(year+1, 1, 1)


In [161]:
from importlib import reload
reload(af_import)
af_import.main(tag_af, af_fnames)

psql -c "CREATE SCHEMA af_mod_na_2012;"

cmd:
ogr2ogr -progress -f PostgreSQL -overwrite PG:dbname='gis' -lco SPATIAL_INDEX=GIST -lco SCHEMA=af_mod_na_2012 -lco GEOMETRY_NAME=geom -lco FID=gid -nln af_in downloads/firms/mod_na_2012/fire_archive_M6_34602.shp

['ogr2ogr', '-progress', '-f', 'PostgreSQL', '-overwrite', "PG:dbname='gis'", '-lco', 'SPATIAL_INDEX=GIST', '-lco', 'SCHEMA=af_mod_na_2012', '-lco', 'GEOMETRY_NAME=geom', '-lco', 'FID=gid', '-nln', 'af_in', 'downloads/firms/mod_na_2012/fire_archive_M6_34602.shp']


## Running "step 1" grouping points

In [162]:
run_step1.main(tag_af, dt0, dt1, ver='v7m')

starting prep: 2019-01-05 00:14:33.771083
starting work 2012-01-01: 2019-01-05 00:14:34.293706
starting work 2012-01-02: 2019-01-05 00:14:34.599112
starting work 2012-01-03: 2019-01-05 00:14:34.817371
starting work 2012-01-04: 2019-01-05 00:14:35.063956
starting work 2012-01-05: 2019-01-05 00:14:35.225865
starting work 2012-01-06: 2019-01-05 00:14:35.443507
starting work 2012-01-07: 2019-01-05 00:14:35.600432
starting work 2012-01-08: 2019-01-05 00:14:35.800524
starting work 2012-01-09: 2019-01-05 00:14:35.948733
starting work 2012-01-10: 2019-01-05 00:14:36.174432
starting work 2012-01-11: 2019-01-05 00:14:36.328936
starting work 2012-01-12: 2019-01-05 00:14:36.538404
starting work 2012-01-13: 2019-01-05 00:14:36.813153
starting work 2012-01-14: 2019-01-05 00:14:37.109943
starting work 2012-01-15: 2019-01-05 00:14:37.363470
starting work 2012-01-16: 2019-01-05 00:14:37.612090
starting work 2012-01-17: 2019-01-05 00:14:37.841301
starting work 2012-01-18: 2019-01-05 00:14:38.104331
star

KeyboardInterrupt: 

## Running "step 2"

In [ ]:
assert run_step2.ver == 'v8b'
rasters = [
        {'tag': tag_lct,
            'kind': 'thematic',
            'variable': 'lct'},
        {'tag': tag_vcf,
            'kind': 'continuous',
            'variables': ['tree', 'herb', 'bare'],
            },
        {'tag': tag_regnum,
            'kind': 'polygons',
            'variable_in': 'region_num',
            'variable': 'regnum',
            }

        ]

In [ ]:
run_step2.main(tag_af, rasters, dt0, dt1)

## Export the output

In [ ]:
odir = '.'
schema = 'af_' + tag_af
tblname = 'out_{0}_{1}_{2}'.format(tag_lct, tag_vcf, tag_regnum)
flds = ('v_lct', 'f_lct', 'v_tree', 'v_herb', 'v_bare', 'v_regnum')
tblname

In [ ]:
export_shp.main(odir, schema, tblname, flds)